# Automatic Grader with Azure OpenAI ChatGPT
This notebook will present the weekly analysis of the student’s final year project log. Students need to submit their progress log every week. The log should answer 4 questions:
1. What did I do this week?
2. What work and findings have I completed since the last log?
3. Which part of the previous plan have I not finished yet? (Leave it blank if I finished all the work from the previous plan.)
4. What is my plan of work before the next log?

### Install packages

In [1]:
%pip install -q pandas openai python-dotenv
%load_ext dotenv
%dotenv

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import urllib.parse

sheet_id = "1V9OH5V8YnflstrZvuiLj-Xdpbeb30Df6V3NjGI3TH2s"
sheet_name = urllib.parse.quote("Form Responses 1")
url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"
df_logs = pd.read_csv(url)

sheet_id = "1qBXDxB9KQrFfgy0AHjiet8kmJ0DoYtIwbgIaWi35orc"
sheet_name = urllib.parse.quote("Name List")
url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"
df_namelist = pd.read_csv(url)

In [3]:
# Filter out student who are in the namelist by Student ID for df_logs
df_logs = df_logs[df_logs["Student ID"].isin(df_namelist["Student No"])]
# Convert Student ID to string without decimal
df_logs["Student ID"] = df_logs["Student ID"].astype(int).astype(str)
df_logs.describe()

,Timestamp,Group ID,Student Name,Student ID,Supervisor,I am doing something this week,Work done & findings since last recording,State which part of the last plan not yet finished (leave it blank if you finished all the work of the last plan,Plan of work before the next recording,Description
count,956,956,956,956,955,956,709,587,687,124
unique,940,15,141,54,8,2,614,366,564,109
top,10/12/2022 9:32:50,A04,Chow Tung Kit,210083096,Cyrus Wong,Yes,proposal,,,
freq,2,86,24,25,383,736,9,65,31,5


In [4]:
#Group by Student ID for df_logs
df_logs_grouped = df_logs.groupby(["Student ID"])
# , group_keys=True).apply(lambda x: x)
df_logs_grouped.head()

,Timestamp,Group ID,Student Name,Student ID,Supervisor,I am doing something this week,Work done & findings since last recording,State which part of the last plan not yet finished (leave it blank if you finished all the work of the last plan,Plan of work before the next recording,Description
0,9/15/2022 10:29:24,A09,ZENG Haoxuan,210107965,Tinson Ngai,Yes,Learn what needs to be done,Project Proposal Just getting started.,Finish Project Proposal,NaN
1,9/21/2022 9:43:49,A04,WONG YIM WAI,210462521,Cyrus Wong,Yes,write the proposal in part2,set up openproject,set up the environmen in cloud,NaN
2,9/21/2022 9:44:15,A03,Ng man hei,210087102,Selena Chiu,Yes,proposal,,power point,NaN
4,9/21/2022 9:45:15,A06,Liu Hok Lai,210241841,Cyrus Wong,Yes,Proposal,PowerPoint,Planning the Cloud Architect,NaN
5,9/21/2022 9:45:36,A03,HUI Man Chun,210396353,Selena Chiu,Yes,write project proposal (situation and proposal...,done,Finish project proposal,NaN
...,...,...,...,...,...,...,...,...,...,...
462,11/23/2022 11:24:16,A11,Ying Ji Kit,210346221,Leo Leung,Yes,try to connect webcam to the virtual character,NaN,NaN,NaN
554,2/3/2023 9:32:18,A15,shum chun yiu,210180843,KH Tong,No,NaN,NaN,NaN,NaN
615,2/10/2023 9:34:47,A15,shum chun yiu,210180843,KH Tong,No,NaN,NaN,NaN,NaN
635,2/10/2023 9:41:00,A12,Chan Wai Ho,210456932,Cyrus Wong,No,NaN,NaN,NaN,NaN


In [5]:
students = {}
for group_name, df_group in df_logs_grouped:
    student_id = group_name[0]
    content = ""
    log_no = 0
    for row_index, row in df_group.iterrows():
        date = row['Timestamp']
        claim_done_something = row['I am doing something this week'] == 'Yes'
        done = row['Work done & findings since last recording']
        unfinished = row['State which part of the last plan not yet finished (leave it blank if you finished all the work of the last plan']
        plan = row['Plan of work before the next recording']

        if str(unfinished) != "nan":
            unfinished= f"""
I did not complete
{unfinished}.
            """
        else:
            unfinished = ""

        log = f"""
Log {log_no} - Weekly log on {date},
I am doing nothing.

"""
        if claim_done_something:
            log = f"""
Log {log_no} - Weekly log on {date},
I completed 
{done}.

{unfinished}

My plan for the next week is
{plan}.

"""
        content += log
        log_no += 1
    students[student_id] = content
        

# for student_id, log in students.items():
#     print(log)
#     print("---------------------------------------------------")

In [6]:
import os
import json
import openai
openai.api_type = "azure"
openai.api_base = os.getenv("AZURE_OPENAI_ENDPOINT") 
openai.api_version = "2023-03-15-preview"
openai.api_key = os.getenv("AZURE_OPENAI_KEY")

def write_text_to_file(path, content):
    with open(path, 'w') as file:
        file.write(content)

def comments_and_marks(student_id,log):    
    prompt = f"""
Act as the student project supervisor, provide feedback based on the following 36 week log:
==================================================    
{log}
==================================================
Student supposes to build a cloud related project and using github.

Rules:
1. Student need done something contantly per week.              20 marks
2. With at least 20 log entries.                                10 marks
3. Show contribution in the project                             20 marks
4. Show progress                                                20 marks
5. Coherence And Cohesion                                       10 marks
6. Done Technical tasks                                         20 marks
7. If a week without any log entry, deduce 1 marks.
8. Students can claim "I am doing nothing." for 6 weeks without any mark deduction.

feebacks:
1. feebacks must not contain double quote!
2. Explain the mark calculation, 
3. Shows the marks from each rule, 
4. Encouraging style here, 
5. Less than 200 words.

"noOfWeekDoingNothing" is the number of weeks that the student did nothing from Sept to April.


Do not include any explanations, only provide a RFC8259 compliant JSON response following this format without deviation.
++++++++++++++++++++++++++++++++++++++++++++++++++
{{
    "marks": 70,
    "noOfWeekDoingNothing": 0,
    "feedback": ""    
}}
++++++++++++++++++++++++++++++++++++++++++++++++++
The JSON response:
"""

    try:
        response = openai.ChatCompletion.create(
            engine="gpt-35-turbo", # engine = "deployment_name".
            messages=[
                {"role": "system", "content": "You are a teaching assistant."},
                {"role": "user", "content": prompt},      
            ]
        )
        # print(prompt)
        # print(response['choices'][0]['message']['content'])
        write_text_to_file(f"tmp/{student_id}.json", json.dumps(response))
        tokens = response['usage']['total_tokens']
        content = json.loads(response['choices'][0]['message']['content'])
        marks = content['marks']
        feedback = content['feedback']        
        noOfWeekDoingNothing = content['noOfWeekDoingNothing']       
        return {
            'student_id':student_id,
            'marks':marks,
            'log':log,
            'feedback': feedback, 
            'noOfWeekDoingNothing':noOfWeekDoingNothing,
            'tokens': tokens, 
            'error': False
            }
    except Exception as e:
        print(e)
        return {
            'student_id':student_id,
            'marks':0,
            'log':log,
            'feedbacks': str(e), 
            'noOfWeekDoingNothing':0,
            'tokens': 0, 
            'error': True
            }

In [7]:
data = []
for student_id, log in students.items():     
    result = comments_and_marks(student_id, log)
    print(result)
    data.append(result)
    # break

df_feedbacks = pd.DataFrame.from_records(data)

{'student_id': '200169933', 'marks': 70, 'log': '\nLog 0 - Weekly log on 9/21/2022 10:07:39,\nI completed \ndo Proposal.\n\n\n\nMy plan for the next week is\nbuild up github.\n\n\nLog 1 - Weekly log on 9/28/2022 9:37:52,\nI completed \nbuild up github team.\n\n\n\nMy plan for the next week is\nfinish Proposal.\n\n\nLog 2 - Weekly log on 10/5/2022 9:59:49,\nI completed \ndone proposal.\n\n\n\nMy plan for the next week is\nPrepare Chatbot content.\n\n\nLog 3 - Weekly log on 10/12/2022 9:45:25,\nI am doing nothing.\n\n\nLog 4 - Weekly log on 10/19/2022 9:47:42,\nI completed \nCheck out some creating gameplay videos to get ready to add some features.\n\n\n\nMy plan for the next week is\nnan.\n\n\nLog 5 - Weekly log on 10/26/2022 9:43:23,\nI am doing nothing.\n\n\nLog 6 - Weekly log on 11/2/2022 9:58:20,\nI completed \nset up cloudwatch.\n\n\n\nMy plan for the next week is\nnan.\n\n\nLog 7 - Weekly log on 11/9/2022 9:58:50,\nI completed \ngame code.\n\n\n\nMy plan for the next week is\nplay

In [9]:
df_feedbacks.to_excel("data/feedback.xlsx", index=False)